In [2]:
import pandas as pd 
import numpy as np 
import os 
import warnings
import random
warnings.filterwarnings("ignore")

data_dir = "data/"

### Mapping column names

In [5]:
field = pd.read_csv(f"{data_dir}field.tsv",sep="\t")
field_dict = dict(zip(field["field_id"], field["title"]))

In [6]:
category = pd.read_csv(f"{data_dir}category.tsv",sep="\t")

In [7]:
category.loc[category['title'].str.contains("Prot")]

,category_id,title,availability,group_type,descript,notes
221,1838,Proteomics,0,1,Proteomics,NaN
222,1839,Protein biomarkers,0,1,~F30900~ in this category grants access to the...,NaN


In [8]:
def rename_columns(df, field_dict):
    # drop instances - take eid only 
    for c in set(df.columns) - {"eid"}:
        df = df.rename(columns={c: c.split("p")[1].split("_")[0]})
        
    # map from eid to name 
    for c in set(df.columns) - {"eid"}:
        df = df.rename(columns={c: field_dict[int(c)]})
        
    return df


In [9]:
def make_unique_col(cols):
    counts = {}
    new_cols = []
    for col in cols:
        if col not in counts:
            counts[col] = 0
            if cols.count(col) > 1:     # only add suffix if it's duplicated
                new_cols.append(f"{col}_{counts[col]}")
            else:
                new_cols.append(col)
        else:
            counts[col] += 1
            new_cols.append(f"{col}_{counts[col]}")
    return new_cols


### Time stamps of labs 

In [7]:
df_lab_time = pd.read_csv(f"{data_dir}time_stamps_participant.csv")
df_lab_time = rename_columns(df_lab_time, field_dict)
df_lab_time = df_lab_time[['eid','Date of attending assessment centre']]

In [8]:
df_lab_time

,eid,Date of attending assessment centre
0,4284267,2007-12-18
1,2846553,2009-08-08
2,5667800,2008-04-04
3,3124022,2009-11-20
4,1986445,2010-07-02
...,...,...
501931,2496856,2007-07-20
501932,4125234,2009-10-16
501933,4450374,2009-03-30
501934,1061049,2009-11-21


### blood count

In [9]:
df_blood_count = pd.read_csv(f"{data_dir}blood_counts_participant.csv")

In [10]:
df_blood_count = rename_columns(df_blood_count, field_dict)

In [11]:
# num of patients 
pd.DataFrame(df_blood_count.count()).reset_index().sort_values(by = 0, ascending = False).head()

,index,0
0,eid,501936
27,Red blood cell (erythrocyte) count,477638
5,Haematocrit percentage,477638
6,Haemoglobin concentration,477638
28,Red blood cell (erythrocyte) distribution width,477636


In [12]:
# num of columns
len(df_blood_count.columns)-1

31

In [13]:
pd.DataFrame(df_blood_count.mean().round(3)).reset_index()

,index,0
0,eid,3510720.889
1,Basophill count,0.034
2,Basophill percentage,0.570
3,Eosinophill count,0.175
4,Eosinophill percentage,2.572
5,Haematocrit percentage,41.085
6,Haemoglobin concentration,14.175
7,High light scatter reticulocyte count,0.018
8,High light scatter reticulocyte percentage,0.402
9,Immature reticulocyte fraction,0.291


### blood chemistry

In [14]:
df_blood_chem = pd.read_csv(f"{data_dir}blood_chem_participant.csv")

In [15]:
df_blood_chem = rename_columns(df_blood_chem, field_dict)

In [16]:
pd.DataFrame(df_blood_chem.count()).reset_index().sort_values(by = 0, ascending = False).head()

,index,0
0,eid,501936
3,Alkaline phosphatase,469075
9,Cholesterol,469062
11,Cystatin C,469031
1,Alanine aminotransferase,468874


In [17]:
# num of columns
len(df_blood_chem.columns)-1

30

In [18]:
pd.DataFrame(df_blood_chem.mean().round(3)).reset_index()

,index,0
0,eid,3510720.889
1,Alanine aminotransferase,23.547
2,Albumin,45.210
3,Alkaline phosphatase,83.674
4,Apolipoprotein A,1.538
5,Apolipoprotein B,1.032
6,Aspartate aminotransferase,26.231
7,C-reactive protein,2.599
8,Calcium,2.380
9,Cholesterol,5.692


### Protein

In [19]:
protein_dir = "data/olink/"
df_olink = pd.read_csv(f"{protein_dir}olink_0.csv")

for i in range(1,6):
    temp = pd.read_csv(f"{protein_dir}olink_{str(i)}.csv")
    df_olink = pd.concat([df_olink, temp], axis=1)
df_olink.columns = df_olink.columns.str.removeprefix('olink_instance_0.')

In [20]:
df_olink

,eid,a1bg,aamdc,aarsd1,abca2,abhd14b,abl1,abo,abraxas2,acaa1,...,zfyve19,zhx2,znf174,znf75d,znf830,znrd2,znrf4,zp3,zp4,zpr1
0,1000083,-0.0499,-0.50515,-1.15870,-1.15675,0.31980,-0.60490,0.3651,-1.4040,0.37400,...,0.2382,-0.2016,1.1164,0.3767,0.00290,-1.38095,-0.3851,1.2104,NaN,0.0754
1,1000380,NaN,NaN,-0.32115,NaN,-0.55405,-0.70055,NaN,NaN,-0.01970,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1001803,-0.1028,-0.06025,-0.70435,-0.39080,-0.56975,0.65230,0.6914,-0.1421,-0.37440,...,-0.3702,-0.2123,-0.1179,0.0615,-0.31760,-1.36705,-0.2538,0.7106,0.5439,-0.1621
3,1002917,-0.0421,-0.94565,-0.52720,0.16580,-0.15770,-0.31360,-2.6708,-0.0256,-0.00570,...,-0.0468,-0.4072,0.1661,0.2073,-0.00495,-1.20780,-0.3506,0.2403,0.1509,-0.8705
4,1003287,0.2050,0.04795,-0.63005,0.44880,-0.42445,-0.05640,-1.7732,-0.4849,0.71205,...,0.0426,0.0014,0.0232,0.4209,-0.57400,0.09655,-0.6307,1.8064,0.3812,-0.0507
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52990,6019370,NaN,NaN,1.34545,NaN,0.52165,-0.60095,NaN,NaN,0.13120,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52991,6019567,-0.0695,0.44265,0.75885,NaN,NaN,0.19280,0.1917,NaN,NaN,...,0.9817,-0.1404,0.0551,-0.0849,2.10030,0.66015,0.2387,1.0304,-0.0938,NaN
52992,6020293,NaN,NaN,-0.39295,NaN,0.85535,-0.12905,NaN,NaN,0.75000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52993,6020560,-0.3638,0.83045,-0.27060,-0.20635,0.20530,1.70000,-1.1375,1.4175,-0.13610,...,0.7644,-0.0486,0.1031,0.3329,0.50050,1.90055,0.2455,1.3979,-0.0309,0.1899


In [21]:
pd.DataFrame(df_olink.count()).reset_index().sort_values(by = 0, ascending = False)

,index,0
0,eid,52995
2265,reg3a,52130
1595,lilrb5,52130
1592,lilrb1,52130
619,cntn1,52130
...,...,...
732,ctss,40459
709,cst1,38868
1991,pcolce,19302
1889,npm1,13785


### Urine assay
Ignore for now

In [22]:
df_urine = pd.read_csv(f"{data_dir}urine_participant.csv")

In [23]:
df_urine = rename_columns(df_urine, field_dict)

In [24]:
df_urine.columns

Index(['eid', 'Creatinine (enzymatic) in urine', 'Microalbumin in urine',
       'Microalbumin in urine result flag', 'Potassium in urine',
       'Sodium in urine'],
      dtype='object')

In [25]:
df_urine['Microalbumin in urine result flag'].value_counts()

Result below 6.7 mg/L    330989
Name: Microalbumin in urine result flag, dtype: int64

In [26]:
df_urine.loc[df_urine['Microalbumin in urine result flag'] == 'Result below 6.7 mg/L', 'Microalbumin in urine'] = 6.7

In [27]:
pd.DataFrame(df_urine.count()).reset_index().sort_values(by = 0, ascending = False).head()

,index,0
0,eid,501936
1,Creatinine (enzymatic) in urine,483765
2,Microalbumin in urine,483745
4,Potassium in urine,482745
5,Sodium in urine,482734


## Demographic and Historical Data

In [7]:
df_hist = pd.read_csv(f"{data_dir}demographic_history_participant.csv")
df_hist = rename_columns(df_hist, field_dict)

In [8]:
df_hist

,eid,Age at recruitment,Sex,Ethnic background,Body mass index (BMI),"Systolic blood pressure, automated reading","Diastolic blood pressure, automated reading",Townsend deprivation index at recruitment,Smoking status,Alcohol intake frequency.,...,Operation code,Operation code,Operation code,Operation code,Operation code,Interpolated Age of participant when operation took place,Interpolated Age of participant when operation took place,Interpolated Age of participant when operation took place,Interpolated Age of participant when operation took place,Interpolated Age of participant when operation took place
0,4284267,65,Female,British,24.7781,152.0,90.0,-3.68,Current,Three or four times a week,...,hysterectomy,bilateral oophorectomy,shoulder surgery,bone surgery/joint surgery,NaN,50.5,50.5,64.8,44.8,NaN
1,2846553,52,Male,British,25.9862,137.0,81.0,-4.14,Never,Once or twice a week,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5667800,63,Male,British,32.0017,145.0,87.0,-0.55,Current,Once or twice a week,...,leg artery angioplasty +/- stent,ear surgery,NaN,NaN,NaN,49.2,36.5,NaN,NaN,NaN
3,3124022,48,Male,British,27.8828,151.0,95.0,-2.77,Previous,One to three times a month,...,vasectomy,NaN,NaN,NaN,NaN,23.9,NaN,NaN,NaN,NaN
4,1986445,63,Female,British,28.2072,109.0,65.0,0.18,Never,Special occasions only,...,appendicectomy,NaN,NaN,NaN,NaN,30.5,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501931,2496856,54,Female,British,37.5381,144.0,89.0,-4.39,Previous,Special occasions only,...,tonsillectomy +/- adenoids,reduction or fixation of bone fracture,NaN,NaN,NaN,21.5,22.5,NaN,NaN,NaN
501932,4125234,41,Male,British,21.8007,119.0,75.0,-2.51,Never,Once or twice a week,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
501933,4450374,64,Male,British,36.4780,106.0,66.0,-1.23,Previous,Three or four times a week,...,appendicectomy,inguinal/femoral hernia repair,retinal operation/vitrectomy,NaN,NaN,3.5,3.5,61.8,NaN,NaN
501934,1061049,45,Female,British,25.0995,122.0,68.0,-2.09,Never,One to three times a month,...,varicose vein surgery,rhinoplasty / nose surgery,NaN,NaN,NaN,28.5,29.5,NaN,NaN,NaN


In [39]:
df_hist['Sex'] = df_hist['Sex'].map({'Male': 1, 'Female': 0})
df_hist = df_hist.rename(columns={'Sex': 'Sex_male'})

## Cancer diagnoses time stamps

In [2]:
df_diag1 = pd.read_csv(f"{data_dir}cancer_type_and_date_participant.csv")
df_diag2 = pd.read_csv(f"{data_dir}cancer_type_and_date_2_participant.csv")
df_diag2 = df_diag2.drop(columns = ['eid'])
df_diag = pd.concat([df_diag1, df_diag2], axis=1)

In [3]:
df_diag = df_diag.rename(columns=lambda x: x.replace('p40005', 'Date of cancer diagnosis'))
df_diag = df_diag.rename(columns=lambda x: x.replace('p40006', 'Type of cancer: ICD10'))

In [7]:
# group ICD
for i in range(22):
    df_diag[[f"diag_{i}_icd10_code", f"diag_{i}_icd10_name"]] = df_diag[f"Type of cancer: ICD10_i{i}"].str.split(" ", n=1, expand=True)
    df_diag[[f"diag_{i}_icd10_grouped", f"diag_{i}_icd10_decimal"]] = df_diag[f"diag_{i}_icd10_code"].str.split(".", n=1, expand=True)

In [8]:
for i in range(22):
    df_diag[f'Type of cancer_i{i}'] = np.nan
    df_diag.loc[(df_diag[f"diag_{i}_icd10_grouped"] == "C50") | (df_diag[f"diag_{i}_icd10_grouped"] == "D05"), f"Type of cancer_i{i}"] = "breast"
    df_diag.loc[(df_diag[f"diag_{i}_icd10_grouped"] == "C61") , f"Type of cancer_i{i}"] = "prostate"
    df_diag.loc[(df_diag[f"diag_{i}_icd10_grouped"] == "C34") , f"Type of cancer_i{i}"] = "lung"
    df_diag.loc[(df_diag[f"diag_{i}_icd10_grouped"] == "C18") | (df_diag[f"diag_{i}_icd10_grouped"] == "C20"), f"Type of cancer_i{i}"] = "colorectal"
    df_diag.loc[(df_diag[f"diag_{i}_icd10_grouped"] == "C22") , f"Type of cancer_i{i}"] = "liver"
    df_diag.loc[(df_diag[f"diag_{i}_icd10_grouped"] == "C25") , f"Type of cancer_i{i}"] = "pancreatic"
    df_diag.loc[(df_diag[f"diag_{i}_icd10_grouped"] == "C67") , f"Type of cancer_i{i}"] = "bladder"

In [ ]:
df_diag

## Merging Data

In [29]:
df_olink = df_olink.add_prefix('olink_')
df_blood_count = df_blood_count.add_prefix('blood_')
df_blood_chem = df_blood_chem.add_prefix('blood_')

df_olink = df_olink.rename(columns = {'olink_eid':'eid'})
df_blood_count = df_blood_count.rename(columns = {'blood_eid':'eid'})
df_blood_chem = df_blood_chem.rename(columns = {'blood_eid':'eid'})

df = pd.merge(df_olink, df_lab_time, on = 'eid', how = 'left')
df = pd.merge(df, df_blood_count, on = 'eid', how = 'left')
df = pd.merge(df, df_blood_chem, on = 'eid', how = 'left')

In [47]:
df = pd.merge(df_hist, df, on='eid', how='right')

In [49]:
# --- 1) Parse dates in df (assessment date) ---
df['assessment_date'] = pd.to_datetime(df['Date of attending assessment centre'], errors='coerce')

# --- 2) Reshape df_diag from wide to long (one row per (eid, checkup)) ---
# This handles columns like "Type of cancer_0", "Date of cancer diagnosis_0", etc.
stubnames = ['Type of cancer', 'Date of cancer diagnosis']
df_diag_long = (
    pd.wide_to_long(df_diag, stubnames=stubnames, i='eid', j='k', sep='_i', suffix=r'\d+')
      .reset_index()
)

# Clean names
df_diag_long = df_diag_long.rename(columns={
    'Type of cancer': 'cancer_type',
    'Date of cancer diagnosis': 'dx_date'
})

# --- 3) Keep valid cancer rows (non-empty type and valid date) ---
df_diag_long['dx_date'] = pd.to_datetime(df_diag_long['dx_date'], errors='coerce')
valid = df_diag_long['cancer_type'].notna() & df_diag_long['dx_date'].notna()
df_diag_long = df_diag_long.loc[valid]

# --- 4) For each (eid, cancer_type), keep the EARLIEST diagnosis date ---
first_dx = (
    df_diag_long
    .groupby(['eid', 'cancer_type'], as_index=False)['dx_date']
    .min()
)

# --- 5) Join assessment date, compute time difference (years) ---
first_dx = first_dx.merge(df[['eid', 'assessment_date']], on='eid', how='left')

first_dx['years_to_dx'] = (first_dx['dx_date'] - first_dx['assessment_date']).dt.days / 365.25
# Cancer present by assessment if diagnosis on/before assessment
first_dx['cancer_flag'] = (first_dx['years_to_dx'] <= 0.083).astype(int)
# If diagnosed before/at assessment, set time_to_diagnosis to 0 (per your rule)
first_dx.loc[first_dx['cancer_flag'] == 1, 'years_to_dx'] = 0.0

# --- 5b) Overall cancer/cancer_time_to_diagnosis from the _i0 column only ---
# Extract and parse the i0 diagnosis date
dx0 = (
    df_diag[['eid', 'Date of cancer diagnosis_i0']]
      .rename(columns={'Date of cancer diagnosis_i0': 'dx0'})
      .assign(dx0=lambda d: pd.to_datetime(d['dx0'], errors='coerce'))
)

# Join assessment date
dx0 = dx0.merge(df[['eid', 'assessment_date']], on='eid', how='left')

# Apply rules:
# - If dx0 is not NA and dx0 <= assessment_date + 30-day buffer: cancer=1
# - If assessment_date < dx0 + 30: cancer=0
present_mask = dx0['dx0'].notna() & (dx0['dx0'] <= dx0['assessment_date'] + pd.Timedelta(days=30))

dx0.loc[present_mask, 'cancer'] = 1
dx0.loc[present_mask, 'cancer_time_to_diagnosis'] = 0.0

# Time to dx in years (positive if dx after assessment, negative if before)
dx0['cancer_time_to_diagnosis'] = (dx0['dx0'] - dx0['assessment_date']).dt.days / 365.25

# If no dx0 date, set cancer=0 and leave time as NaN
dx0['cancer'] = dx0['cancer'].fillna(0).astype(int)

# Keep only needed columns for merge
dx0_overall = dx0[['eid', 'cancer', 'cancer_time_to_diagnosis']]


# --- 6) Pivot back to wide: one column per cancer type ---
flags_wide = (
    first_dx.pivot(index='eid', columns='cancer_type', values='cancer_flag')
            .add_suffix('_cancer')
)
times_wide = (
    first_dx.pivot(index='eid', columns='cancer_type', values='years_to_dx')
            .add_suffix('_time_to_diagnosis')
)

# Ensure ints for flags, fill missing (no record) with 0 for flags
flags_wide = flags_wide.fillna(0).astype(int)

# --- 7) Merge onto df ---
df_out = (
    df.merge(flags_wide, on='eid', how='left')
      .merge(times_wide, on='eid', how='left')
      .merge(dx0_overall, on='eid', how='left')
)
df_out = df_out.drop(columns=['Date of attending assessment centre'])


In [59]:
for k in ['breast_cancer', 'prostate_cancer', 'lung_cancer', 'colorectal_cancer', 'liver_cancer', 'bladder_cancer', 'pancreatic_cancer']:
    df_out[k] = df_out[k].fillna(0)
    df_out[k] = df_out[k].astype(int)

In [55]:
## after adding 30 day buffer
for k in ["breast", "prostate", "lung", "colorectal", "bladder", "pancreatic", "liver"]:
    print(k, df_out[f'{k}_cancer'].value_counts())
    print()
    
print("cancer", df_out['cancer'].value_counts())

breast 0    51989
1     1006
Name: breast_cancer, dtype: int64

prostate 0    52648
1      347
Name: prostate_cancer, dtype: int64

lung 0    52951
1       44
Name: lung_cancer, dtype: int64

colorectal 0    52775
1      220
Name: colorectal_cancer, dtype: int64

bladder 0    52950
1       45
Name: bladder_cancer, dtype: int64

pancreatic 0    52987
1        8
Name: pancreatic_cancer, dtype: int64

liver 0    52992
1        3
Name: liver_cancer, dtype: int64

cancer 0    48037
1     4958
Name: cancer, dtype: int64


In [58]:
df_out.loc[df_out['cancer_time_to_diagnosis']<=5]

,eid,Age at recruitment,Sex_male,Ethnic background,Body mass index (BMI),"Systolic blood pressure, automated reading","Diastolic blood pressure, automated reading",Townsend deprivation index at recruitment,Smoking status,Alcohol intake frequency.,...,prostate_cancer,bladder_time_to_diagnosis,breast_time_to_diagnosis,colorectal_time_to_diagnosis,liver_time_to_diagnosis,lung_time_to_diagnosis,pancreatic_time_to_diagnosis,prostate_time_to_diagnosis,cancer,cancer_time_to_diagnosis
0,1000083,49,0,British,24.7295,116.0,71.0,-3.96,Previous,Three or four times a week,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1.886379
12,1005250,52,0,British,20.5859,159.0,89.0,-5.24,Never,Special occasions only,...,0,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,1,-1.032170
20,1009427,44,0,British,21.7900,107.0,68.0,-5.65,Never,Special occasions only,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,-4.391513
22,1010774,50,1,British,23.3188,158.0,76.0,5.07,Current,Once or twice a week,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2.258727
24,1011206,60,1,British,28.8009,119.0,76.0,-1.34,Previous,Daily or almost daily,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,4.648871
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52959,6005390,69,0,British,21.8620,166.0,84.0,2.35,Previous,Once or twice a week,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2.179329
52963,6007500,60,0,Any other white background,26.0670,137.0,69.0,3.41,Never,Once or twice a week,...,0,NaN,10.362765,NaN,NaN,NaN,NaN,NaN,1,-7.556468
52977,6016461,54,0,Pakistani,31.7694,169.0,84.0,2.84,Never,Never,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1.856263
52978,6016558,69,1,British,27.6528,142.0,80.0,-1.61,Previous,Once or twice a week,...,0,NaN,NaN,0.210815,NaN,NaN,NaN,NaN,0,0.210815


In [60]:
df_out.to_csv(f"{data_dir}blood_protein_cancers_clean.csv", index = False)

## Cancer exploration
Get statistics of the cancers in the dataset

In [29]:
df = pd.read_csv(f"{data_dir}blood_protein_cancers_clean.csv")
df_cancer = pd.merge(df, df_diag[['eid','Type of cancer: ICD10_i0']], on='eid', how='left')

In [31]:
df_cancer.to_csv(f"{data_dir}blood_protein_cancers_clean_with_icd10.csv", index = False)

In [ ]:
df_with_cancer = df_cancer[df_cancer['diag_0_icd10_grouped'].notna()]

In [26]:
df_with_cancer_counts = df_with_cancer['Type of cancer: ICD10_i0'].value_counts()

In [ ]:
df_with_cancer_counts

C44.9 Malignant neoplasm of skin, unspecified                                          1625
C61 Malignant neoplasm of prostate                                                     1415
C44.3 Skin of other and unspecified parts of face                                       742
C50.9 Breast, unspecified                                                               703
C50.4 Upper-outer quadrant of breast                                                    448
                                                                                       ... 
C00.1 External lower lip                                                                  1
C43.1 Malignant melanoma of eyelid, including canthus                                     1
C42.1 hematopoietic and reticuloendothelial systems: bone marrow (ICD-O-3 specific)       1
C14.0 Pharynx, unspecified                                                                1
C78.0 Secondary malignant neoplasm of lung                                      

In [32]:
df_cancer = pd.merge(df, df_diag, on='eid', how='left')

In [37]:
temp = df_cancer.loc[df_cancer['diag_0_icd10_grouped']=='D05']

In [46]:
temp = temp.iloc[:,-150:-130]

In [48]:
temp[temp['Type of cancer: ICD10_i1'].notna()]

,Date of cancer diagnosis_i4,Date of cancer diagnosis_i5,Type of cancer: ICD10_i0,Type of cancer: ICD10_i1,Type of cancer: ICD10_i2,Type of cancer: ICD10_i3,Type of cancer: ICD10_i4,Type of cancer: ICD10_i5,Type of cancer: ICD10_i6,Type of cancer: ICD10_i7,Type of cancer: ICD10_i8,Type of cancer: ICD10_i9,Type of cancer: ICD10_i10,Type of cancer: ICD10_i11,Type of cancer: ICD10_i12,Type of cancer: ICD10_i13,Type of cancer: ICD10_i14,Type of cancer: ICD10_i15,Type of cancer: ICD10_i16,Type of cancer: ICD10_i17
892,NaN,NaN,D05.1 Intraductal carcinoma in situ,C50.4 Upper-outer quadrant of breast,"D05.9 Carcinoma in situ of breast, unspecified",C50.4 Upper-outer quadrant of breast,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1083,NaN,NaN,D05.1 Intraductal carcinoma in situ,"C50.9 Breast, unspecified",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2512,NaN,NaN,D05.1 Intraductal carcinoma in situ,C50.8 Overlapping lesion of breast,C50.8 Overlapping lesion of breast,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2665,NaN,NaN,D05.0 Lobular carcinoma in situ,"C44.9 Malignant neoplasm of skin, unspecified",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2770,NaN,NaN,D05.1 Intraductal carcinoma in situ,"C50.9 Breast, unspecified",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49875,NaN,NaN,D05.1 Intraductal carcinoma in situ,"D05.9 Carcinoma in situ of breast, unspecified",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51384,NaN,NaN,D05.1 Intraductal carcinoma in situ,"C50.9 Breast, unspecified","C67.9 Bladder, unspecified",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51798,NaN,NaN,D05.0 Lobular carcinoma in situ,C50.4 Upper-outer quadrant of breast,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51959,NaN,NaN,D05.1 Intraductal carcinoma in situ,C54.1 Endometrium,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Train/val/test split of cancer dataset
1. Drop protein columns that have over 30% missing
2. Split into train and test using iterative-stratification on cancer + top 7 cancers (with each time frame)

In [18]:
df = pd.read_csv('data/blood_protein_cancers_clean.csv')
# Select only columns that start with "olink"
olink_cols = [col for col in df.columns if col.startswith("olink")]

# Compute the fraction of missing olink values per row
row_missing_fraction = df[olink_cols].isna().mean(axis=1)

# Filter out rows with >30% missing olink values
rows_to_drop = df[row_missing_fraction >= 0.3].index

preprocessed_df = df[row_missing_fraction < 0.3]

# Compute the fraction of missing values for these columns
missing_fraction = preprocessed_df[olink_cols].isna().mean()

# Keep only columns with ≤ 30% missing values
cols_to_drop = missing_fraction[missing_fraction >= 0.3].index # only 3 columns

# Drop the columns with >30% missing
preprocessed_df = preprocessed_df.drop(columns=[col for col in olink_cols if col in cols_to_drop])

In [19]:
print(f"dropping patients: {len(rows_to_drop)}")
print(f"dropping cols: {len(cols_to_drop)}")

dropping patients: 8232
dropping cols: 3


In [57]:
# Compute the fraction of missing olink values per row
row_missing_fraction = df[olink_cols].isna().mean(axis=1)

# Filter out rows with >30% missing olink values
rows_to_drop = df[row_missing_fraction >= 0.7]

In [42]:
# Create decile bins
bins = [i/10 for i in range(11)]  # [0.0, 0.1, 0.2, ..., 1.0]
labels = [f"{bins[i]:.1f}-{bins[i+1]:.1f}" for i in range(10)]

# Bin the missing fractions
bucketed = pd.cut(row_missing_fraction, bins=bins, labels=labels, include_lowest=True, right=False)

# Count number of rows per bucket
bucket_counts = bucketed.value_counts().sort_index()

print(bucket_counts)

0.0-0.1    41383
0.1-0.2     2778
0.2-0.3      602
0.3-0.4      121
0.4-0.5     1406
0.5-0.6     6307
0.6-0.7      249
0.7-0.8       71
0.8-0.9       67
0.9-1.0       11
dtype: int64


In [58]:
sum(rows_to_drop['cancer_time_to_diagnosis'].notna())

40

In [8]:
cols_to_drop

Index(['olink_glipr1', 'olink_npm1', 'olink_pcolce'], dtype='object')

In [23]:
label_time_cols = [
    "cancer",
    "breast",
    "prostate",
    "lung",
    "colorectal",
    "bladder", 
    "pancreatic"
]

def bin_ttd(x):
    if pd.isna(x):         return "NA"
    if x <= 30/365.25:     return "<0"
    if 30/365.25 < x <= 1: return "0-1"   # (0,1]
    if 1 < x <= 5:         return "1-5"   # (1, 5]
    return ">5"

def proportions(frame, label):
    return (frame[f"{label}_strata"].value_counts(normalize=False)
            .reindex(["<0","0-1","1-5",">5","NA"])
            .fillna(0))

In [24]:
for label in label_time_cols:
    preprocessed_df[f"{label}_strata"] = preprocessed_df[f"{label}_time_to_diagnosis"].apply(bin_ttd)
    print(f"\n{label}:\n", proportions(preprocessed_df, label))


cancer:
 <0      4139
0-1      454
1-5     2179
>5      4280
NA     33711
Name: cancer_strata, dtype: int64

breast:
 <0       857
0-1       75
1-5      332
>5       560
NA     42939
Name: breast_strata, dtype: int64

prostate:
 <0       288
0-1       64
1-5      310
>5       722
NA     43379
Name: prostate_strata, dtype: int64

lung:
 <0        40
0-1       28
1-5      138
>5       310
NA     44247
Name: lung_strata, dtype: int64

colorectal:
 <0       188
0-1       36
1-5      145
>5       364
NA     44030
Name: colorectal_strata, dtype: int64

bladder:
 <0        36
0-1        8
1-5       33
>5        81
NA     44605
Name: bladder_strata, dtype: int64

pancreatic:
 <0         5
0-1        7
1-5       30
>5        85
NA     44636
Name: pancreatic_strata, dtype: int64


In [27]:
for label in label_time_cols:
    df[f"{label}_strata"] = df[f"{label}_time_to_diagnosis"].apply(bin_ttd)
    print(f"\n{label}:\n", proportions(df, label))


cancer:
 <0      4958
0-1      526
1-5     2582
>5      5138
NA     39791
Name: cancer_strata, dtype: int64

breast:
 <0      1006
0-1       87
1-5      385
>5       666
NA     50851
Name: breast_strata, dtype: int64

prostate:
 <0       347
0-1       79
1-5      373
>5       871
NA     51325
Name: prostate_strata, dtype: int64

lung:
 <0        44
0-1       30
1-5      156
>5       362
NA     52403
Name: lung_strata, dtype: int64

colorectal:
 <0       220
0-1       42
1-5      179
>5       441
NA     52113
Name: colorectal_strata, dtype: int64

bladder:
 <0        45
0-1        9
1-5       44
>5        99
NA     52798
Name: bladder_strata, dtype: int64

pancreatic:
 <0         8
0-1        9
1-5       34
>5       102
NA     52842
Name: pancreatic_strata, dtype: int64


In [ ]:
## Create train, validation, and test datasets for predicting current cancer
from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit
# Sanity check: compare prevalences per label in full vs train vs test
def prevalence(table, cols):
    return pd.DataFrame({
        "prevalence": [table[c].mean() for c in cols],
        "n": [table[c].sum() for c in cols]
    }, index=cols)

def multilabel_stratified_split(
    df: pd.DataFrame,
    test_size=0.4,
    random_state=42,
    time=0
):
    """
    Splits df into train/test so that each label in label_cols
    has (approximately) the same prevalence in both splits,
    accounting for multi-label rows.
    """
    # 1) Build the multi-label target matrix (n_samples x n_labels)
    # Y = df[label_cols].astype(int).to_numpy()
    df_copy = df.copy()
    Y_cols = []
    for label in label_time_cols:
        for strata in ["<0","0-1","1-5",">5"]:
            df_copy[f"{label}_time_to_diagnosis_{strata}"] = df[f"{label}_strata"] == strata
            Y_cols.append(f"{label}_time_to_diagnosis_{strata}")
    Y = df_copy[Y_cols].to_numpy()

    # 2) Set up the multi-label stratified splitter
    msss = MultilabelStratifiedShuffleSplit(
        n_splits=1, test_size=test_size, random_state=random_state
    )

    # 3) Run the split; indices refer to rows of df
    (train_idx, test_idx), = msss.split(df, Y)

    train_df = df.iloc[train_idx].copy()
    test_df  = df.iloc[test_idx].copy()
    
    # train_df = df_copy.iloc[train_idx].copy()
    # test_df  = df_copy.iloc[test_idx].copy()
    
    # summary = pd.concat(
    #     {
    #         "full": prevalence(df_copy, Y_cols),
    #         "train": prevalence(train_df, Y_cols),
    #         "test": prevalence(test_df, Y_cols),
    #     },
    #     axis=1,
    # )
    # print(summary)

    return train_df, test_df

# Make the split
train_df, validtest_df = multilabel_stratified_split(df, test_size=0.4)
valid_df, test_df = multilabel_stratified_split(validtest_df, test_size=0.5)

train_df.to_csv("data/ukb_cancer_train.csv", index=False)
valid_df.to_csv("data/ukb_cancer_valid.csv", index=False)
test_df.to_csv("data/ukb_cancer_test.csv", index=False)

## Add follow up dates

In [41]:
data_path = "/orcd/pool/003/dbertsim_shared/ukb"

df_lab_time = rename_columns(pd.read_csv(f"data/time_stamps_participant.csv"),field_dict)
df_assessment_centre = rename_columns(pd.read_csv(f"data/assessment_centre.csv"),field_dict)
df = pd.merge(df_lab_time, df_assessment_centre, how = 'left', on = 'eid')

ASSESSMENT_CENTRE_TO_COUNTRY = {
    # England
    "Barts": "England",
    "Birmingham": "England",
    "Bristol": "England",
    "Bury": "England",
    "Cheadle (revisit)": "England",
    "Croydon": "England",
    "Hounslow": "England",
    "Leeds": "England",
    "Liverpool": "England",
    "Manchester": "England",
    "Middlesborough": "England",
    "Newcastle": "England",
    "Nottingham": "England",
    "Oxford": "England",
    "Reading": "England",
    "Sheffield": "England",
    "Stockport (pilot)": "England",
    "Stoke": "England",
    "Cheadle (imaging)": "England",
    "Reading (imaging)": "England",
    "Newcastle (imaging)": "England",
    "Bristol (imaging)": "England",

    # Scotland
    "Edinburgh": "Scotland",
    "Glasgow": "Scotland",

    # Wales
    "Cardiff": "Wales",
    "Swansea": "Wales",
    "Wrexham": "Wales",
}
COUNTRY_CENSOR_DATE = {
    "England":  pd.Timestamp("2023-05-31"),
    "Scotland": pd.Timestamp("2023-09-30"),
    "Wales":    pd.Timestamp("2016-12-31"),
}
df["country"] = df["UK Biobank assessment centre"].map(ASSESSMENT_CENTRE_TO_COUNTRY)
df["admin_censor_date"] = df["country"].map(COUNTRY_CENSOR_DATE)
df['admin_censor_date'] = pd.to_datetime(df['admin_censor_date'])
# if nan, then assume earlier date (2016)
df.loc[df['admin_censor_date'].isna(), 'admin_censor_date'] = pd.Timestamp("2016-12-31")

df['Date of attending assessment centre'] = pd.to_datetime(df['Date of attending assessment centre'])

df['time_to_follow_up'] = (df["admin_censor_date"] - df["Date of attending assessment centre"]).dt.days / 365.25


df_train = pd.read_csv(f"{data_path}/ukb_cancer_train.csv")
df_valid = pd.read_csv(f"{data_path}/ukb_cancer_valid.csv")
df_test = pd.read_csv(f"{data_path}/ukb_cancer_test.csv")

df_train = pd.merge(df_train, df[['eid','time_to_follow_up']], how = 'left', on = 'eid')
df_valid = pd.merge(df_valid, df[['eid','time_to_follow_up']], how = 'left', on = 'eid')
df_test = pd.merge(df_test, df[['eid','time_to_follow_up']], how = 'left', on = 'eid')

df_train = df_train.drop(columns = ['cancer_strata','breast_strata','prostate_strata','lung_strata','colorectal_strata','bladder_strata','pancreatic_strata'])
df_valid = df_valid.drop(columns = ['cancer_strata','breast_strata','prostate_strata','lung_strata','colorectal_strata','bladder_strata','pancreatic_strata'])
df_test = df_test.drop(columns = ['cancer_strata','breast_strata','prostate_strata','lung_strata','colorectal_strata','bladder_strata','pancreatic_strata'])

df_train.to_csv(f"{data_path}/ukb_cancer_train.csv", index=False)
df_valid.to_csv(f"{data_path}/ukb_cancer_valid.csv", index=False)
df_test.to_csv(f"{data_path}/ukb_cancer_test.csv", index=False)
